# Slide-Seq Mouse Olfactory Bulb - single puck

This example uses TACCO to annotate and analyse mouse olfactory bulb Slide-Seq data (Wang et al.) with mouse olfactory bulb scRNA-seq data (Tepe et al.) as reference.

(Wang et al.): Wang IH, Murray E, Andrews G, Jiang HC et al. Spatial transcriptomic reconstruction of the mouse olfactory glomerular map suggests principles of odor processing. Nat Neurosci 2022 Apr;25(4):484-492. PMID: 35314823

(Tepe et al.): Tepe B, Hill MC, Pekarek BT, Hunt PJ et al. Single-Cell RNA-Seq of Mouse Olfactory Bulb Reveals Cellular Heterogeneity and Activity-Dependent Molecular Census of Adult-Born Neurons. Cell Rep 2018 Dec 4;25(10):2689-2703.e3. PMID: 30517858

In [ ]:
import os
import sys

import pandas as pd
import numpy as np
import anndata as ad

import tacco as tc

# The notebook expects to be executed either in the workflow directory or in the repository root folder...
sys.path.insert(1, os.path.abspath('workflow' if os.path.exists('workflow/common_code.py') else '..')) 
import common_code

## Load data

In [ ]:
data_path = common_code.find_path('results/slideseq_mouse_olfactory_bulb')

In [ ]:
reference = ad.read(f'{data_path}/reference.h5ad')

In [ ]:
puck = ad.read(f'{data_path}/puck_1_5.h5ad')

## Get a first impression of the spatial data

Plot total counts

In [ ]:
puck.obs['total_counts'] = tc.sum(puck.X,axis=1)
puck.obs['log10_counts'] = np.log10(1+puck.obs['total_counts'])

In [ ]:
fig = tc.pl.scatter(puck, 'log10_counts', cmap='viridis', cmap_vmin_vmax=[1,3]);
fig.savefig(f'{data_path}/scatter_counts_single.pdf',bbox_inches='tight')

Plot marker genes

In [ ]:
cluster2type = reference.obs[['ClusterName','type']].drop_duplicates().groupby('type')['ClusterName'].agg(lambda x: list(x.to_numpy()))
type2long = reference.obs[['type','long']].drop_duplicates().groupby('long')['type'].agg(lambda x: list(x.to_numpy()))

In [ ]:
marker_map = {}
for k,v in type2long.items():
    genes = []
    if '(' in k:
        genes = k.split('(')[-1].split(')')[0].split('/')
    marker_map[v[0]] = [g[:-1] for g in genes]

In [ ]:
puck.obsm['type_mrk'] = pd.DataFrame(0.0, index=puck.obs.index, columns=sorted(reference.obs['type'].unique()))
for k,v in marker_map.items():
    for g in v:
        puck.obsm['type_mrk'][k] += puck[:,g].X.A.flatten()
    total = puck.obsm['type_mrk'][k].sum()
    if total > 0:
        puck.obsm['type_mrk'][k] /= total

In [ ]:
fig=tc.pl.scatter(puck, 'type_mrk', compositional=True);
fig.savefig(f'{data_path}/scatter_marker_single.pdf',bbox_inches='tight')

## Annotate the spatial data with compositions of cell types

Annotation is done on cluster level to capture variation within a cell type...

In [ ]:
tc.tl.annotate(puck,reference,'ClusterName',result_key='ClusterName',)

... and then aggregated to cell type level for visualization

In [ ]:
tc.utils.merge_annotation(puck, 'ClusterName', cluster2type, 'type');
tc.utils.merge_annotation(puck, 'type', type2long, 'long');

In [ ]:
fig = tc.pl.scatter(puck, 'type', );
fig.savefig(f'{data_path}/scatter_type_single.pdf',bbox_inches='tight')

## Analyse co-occurrence and neighbourhips

Calculate distance matrices per sample and evaluate different spatial metrics on that. Using sparse distance matrices is useful if one is interested only in small distances relative to the sample size.

In [ ]:
tc.tl.co_occurrence(puck, 'type', result_key='type-type',delta_distance=20,max_distance=1000,sparse=False,n_permutation=10)

In [ ]:
fig = tc.pl.co_occurrence(puck, 'type-type', log_base=2, wspace=0.25);
fig.savefig(f'{data_path}/cooc_line_single.pdf',bbox_inches='tight')

In [ ]:
fig = tc.pl.co_occurrence_matrix(puck, 'type-type', score_key='z', restrict_intervals=[0,1,3,7],cmap_vmin_vmax=[-5,5], value_cluster=True, group_cluster=True);
fig.savefig(f'{data_path}/cooc_matrix_single.pdf',bbox_inches='tight')